# Lab 04: YOLO (v3 and v7)

In this lab, we'll explore a fascinating use of image classification deep neural networks to peform a different
task: object detection.

Credits: parts of this lab are based on other authors' code and blog posts:

- YOLO v3 in PyTorch: [tutorial code](https://github.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch),
  [blog](https://blog.paperspace.com/how-to-implement-a-yolo-object-detector-in-pytorch/)
- [YOLO v7 explained](https://blog.roboflow.com/yolov7-breakdown/)


## Object Detection

If you could go back in time to the 1990s, there were no cameras that could find faces in a photograph, and no
researcher had a way to count dogs in a video in real time. Everyone had to count the dogs manually.
Times were very tough.

The Holy Grail of computer vision research at the time was real time face detection. If we could find faces
in images fast enough, we could build systems that interact more naturally with human beings. But nobody had
a solution.

Things changed when Viola and Jones introduced the first real time face detector, the Haar-like cascade, at
the end of the 1990s.
This technique swept a detection window over the input image at multiple sizes, and subjected each local patch
to a cascade of simple rough classifiers. Each patch that made it to the end of the cascade of classifiers was
treated as a positive detection. After a set of candidate patches were identified, there would be a cleanup
stage when neighboring detections are clustered into isolated detections.

This method and one cousin, the HOG detector, which was slower but a little more accurate, dominated during the 2000s
and on into the 2010s. These methods worked well enough when trained carefully on the specific environment they were
used in, but usually couldn't be transfer to a new environment.

With the introduction of AlexNet and the amazing advances in image classification, we could follow the direction
of R-CNN, to use a region proposal algorithm followed by a deep learning classifier to do object detection VERY slowly
but much more accurately than the old real time methods.

## What is YOLO?

However, it wasn't until YOLO that we had a deep learning model for object detection that could run in real time.
It took some clever insight to realize that everything, from feature extraction to bounding box estimation, could
actually be done with a single neural network that could be trained end-to-end to detect objects.

YOLO (You Only Look Once) uses only convolutional layers. This makes it a "fully convolutional network" or FCN.

YOLOv3 has 75 convolutional layers, with skip connections and upsampling layers. No pooling is used, but there is a convolutional
layer with stride 2 used for downsampling. Strided convoution rather than pooling was used to prevent loss of fine-grained detailed
information about the precise location of low-level features that would otherwise occur with pooling.

Normally, the output of a convolutional layer is a feature map. Applying convolutional layers to a possible detection window or
region of interest (ROI) in the image then classifying the ROI's feature map is a reasonable method for detection prediction that
is used in Fast R-CNN and Faster R-CNN.
However, the innovation of YOLO was to use the feature map directly to predict bounding boxes and, for each bounding box, to
predict whether or not an object is at the center of the bounding box. Finally, a classifier is used for each bounding box
to indicate the content of the bounding box.

## YOLO v3 from "scratch"

Early versions of YOLO were very fast but not nearly as accurate as their slower cousins. YOLO v3 included many of the
tricks and techniques used by other models, such as multiscale analysis, and it achieved both high accuracy and fast inference.

Here we'll experiment with building up the YOLO v3 model in PyTorch. However, we won't train it ourselves, as that would
require days of training; instead, we'll
grab the weights for our PyTorch YOLO v3 from the original Darknet model by Joseph Redmon and friends.

### Ground Truth Bounding Boxes

Here is how we present example images and corresponding object bounding boxes to the model.

The input image is divided into grid cells. The number of cells depends on the number of convolutional layers
and the stride of each of those convolutional layers. For example, if we use a 416$\times$416 input image size,
and we apply 5 conv layers with a stride of 2 each (for a total downsampling factor of 32), we end up with a 13$\times$13
feature map, each corresponding to a region in the original image of size 32$\times$32 pixels.

A ground truth box has a center (x and y position), a width, and a height. Normally the ground truth boxes would be
provided by a human annotator.

Each ground truth box's center must lie in some grid cell in the original image. Consider this example from the YOLO paper:

<img src="img/yolo05.png" title="GroundTruthBox" style="width: 400px;" />

The grid is represented by the black lines. The ground truth bounding box for the object is the yellow rectangle. The center
of this bounding box happens to be within the red-outlined grid cell.

The grid cell containing the center of a ground truth bounding box is given the responsibility during training to try to predict
the presence of the object.

In order to indicate the presence of the given object, the model outputs several parameters for a given candidate object:
 - $(t_x, t_y, t_w, t_h)$ indicate the box's location and size. During training, the targets for these outputs are the actual ground truth box parameters.
 - $p_o$ is an "objectness" score that indicates the likelihood that an object exists in the given bounding box. This output uses a sigmoid function.
   During training, the target for $p_o$ is set to 1 for the center grid cell (the red grid cell), and it is set to 0 for the the neighboring grid cells.
 - $(p_1, p_2, \ldots, p_n)$ are class confidence scores. They indicate the probability of the detected object belonging to a particular class. The targets,
   obviously, are set to 1 for the ground truth object class and 0 for other classes during training.

### Anchor Boxes

One problem that would occur in YOLO if you tried to directly learn the parameters mentioned above is the problem of unstable gradients during training.
In a way that is sort of analagous to how a residual block begins with an identity map and learns differences from identity, YOLO v3 uses the idea of
anchor boxes originally introduced by the R-CNN team. Instead of predicting $(t_x, t_y, t_w, t_h)$ directly, we predict how those parameters are *different from
the parameters of a typical bounding box, an anchor box*.
YOLO v3 uses three bounding boxes per cell. At training time, once ground truth bounding box's center is mapped to a grid cell, we find which of the anchors for
that cell has the highest IoU with the ground truth box.

### So What Does YOLO Actually Predict?

First, let's understand that all predictions are relative to the grid cell. YOLO predicts the following:
- Offsets $(t_x, t_y)$ are specified relative to the top left corner of the grid cell, as a ratio between 0 and 1, using a sigmoid to limit the values.
- Height, and width $(t_w, t_h)$ are specified relative to the dimensions of an anchor box.

Thus, YOLO does not predict absolute coordinates -- it predicts values that can then be used to compute the box's position and size in absolute coordinates.
This diagram gives the idea. We see that the absolute $t_x$ is the grid cell's $(c_x, c_y)$ plus $\sigma(t_x)$ times the grid cell width. Similarly for $t_y$.
The absolute width of the predicted bounding box is the width of the anchor box times $e^{tw}$. Similarly for the height.

<img src="img/yolo06.png" title="GroundTruthBox" style="width: 640px;" />

Hopefully you can see the difference between the YOLO v3 bounding box predictions and the Faster R-CNN bounding box predictions. The offset of the center is
encoded relative to the grid cell containing the anchor box rather than the anchor box itself. The dimensions of the bounding box, however, similar to Faster R-CNN,
are predicted relative to the anchor box size.

### Multi-scale prediction

Rather than a single grid size and grid cell size,
YOLO v3 detects objects at multiple sizes with downsampling factors of 32, 16, and 8. The largest objects are detected at the
first, coarsest scale, whereas mid-sized objects are detected at the intermediate scale, and small objects are detected at the finest
scale. The example below shows the three grid sizes relative to the image and an object:

<img src="img/yolo_Scales.png" title="GroundTruthBox" style="width: 640px;" />

### YOLO dataset format

A YOLO dataset contains two sets of files, each pair with the same name: image files (in any supported format)
and label files (in TXT, JSON, or XML format). A label file contains data in the format

$$i, C_x, C_y, L_x, L_y$$

- $i$: Label index
- $C_x$: Center position in the horizontal ($x$-axis) direction, encoded in the range 0-1, where 0 means the left edge of the image and 1 means the right edge.
- $C_y$: Center position in the vertical ($y$-axis) direction, encoded in the range 0-1, where 0 means the top edge of the image and 1 means the bottom edge.
- $L_x$: Object width, encoded in the range 0-1, where 1 means the width of the image.
- $L_y$: Object height, encoded in the range 0-1, where 1 means the height of the image.

To calculate these values for an object, suppose $(W,H)$ is the actual size of a particular image in pixels,
$(O_x, O_y)$ is the actual position of an object in that image, in pixels, and $(l_x,l_y)$ is the actual size of the object, again in pixels.
The object label elements would be calculated as

$$C_x = \frac{O_x}{W}, \; C_y = \frac{O_y}{H} \\
L_x = \frac{l_x}{W}, \; L_y = \frac{l_y}{H}$$

### YOLOv3 Architecture

<img src="img/YOLOv3_Architecture.png" title="YOLOv3" style="width: 960px;" />

### Preparation for Building YOLO in PyTorch

First of all, we will need OpenCV:

    pip3 install --upgrade pip
    pip install matplotlib opencv-python

Create a directory where the code for your detector will live.

In that directory, download util.py and darknet.py from https://github.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch.

In Jupyter you would download thusly:

In [1]:
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/darknet.py
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/util.py

--2021-02-06 00:55:49--  https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/darknet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11533 (11K) [text/plain]
Saving to: ‘darknet.py’

darknet.py          100%[===================>]  11.26K  --.-KB/s    in 0s      

2021-02-06 00:55:50 (80.7 MB/s) - ‘darknet.py’ saved [11533/11533]

--2021-02-06 00:55:50--  https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/util.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7432 (7.3K) [text/plain]
Saving to: ‘util.py’

util.py             100%[==============

If you're in a Docker container, just run the `wget` commands at the command line. Make sure your proxy environment variables are set correctly.

### Take a Look at the YOLO Darknet Configuration File

Next, let's download the `yolov3.cfg` configuration file and take a look. You could grab it from the canonical Darknet github repository
or any other place it's stored.

In [2]:
!mkdir -p cfg
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/cfg/yolov3.cfg
!mv yolov3.cfg cfg/yolov3.cfg

--2021-02-06 00:55:57--  https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8346 (8.2K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2021-02-06 00:55:58 (23.4 MB/s) - ‘yolov3.cfg’ saved [8346/8346]



The configuration file looks like this:

    [net]
    # Testing
    batch=1
    subdivisions=1
    # Training
    # batch=64
    # subdivisions=16
    width= 416

    height = 416
    channels=3
    momentum=0.9
    decay=0.0005
    angle=0
    saturation = 1.5
    exposure = 1.5
    hue=.1

    learning_rate=0.001
    burn_in=1000
    max_batches = 500200
    policy=steps
    steps=400000,450000
    scales=.1,.1

    [convolutional]
    batch_normalize=1
    filters=32
    size=3
    stride=1
    pad=1
    activation=leaky

    ...

    [shortcut]
    from=-3
    activation=linear

    ...

    [yolo]
    mask = 6,7,8
    anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
    classes=80
    num=9
    jitter=.3
    ignore_thresh = .7
    truth_thresh = 1
    random=1

    [route]
    layers = -4

    [convolutional]
    batch_normalize=1
    filters=256
    size=1
    stride=1
    pad=1
    activation=leaky

    [upsample]
    stride=2

    [route]
    layers = -1, 61

    ...



### Overview of the Configuration Blocks

The configuration blocks fall into a few cateogies:

- Net: the global configuration at the top of the configuration file. It declares the size of input images, batch size, learning rate, and so on.

      batch=64
      subdivisions=16
      width=608
      height=608
      channels=3
      momentum=0.9
      decay=0.0005
      angle=0
      saturation = 1.5
      exposure = 1.5
      hue=.1


- Convolutional: convolutional layer. Note that this specfication is a little more powerful than the PyTorch way of doing things, as options
  for batch normalization and the activation function (leaky ReLU in this case) are built in.
 
      [convolutional]
      batch_normalize=1
      filters=32
      size=3
      stride=1
      pad=1
      activation=leaky
        

- Shortcut: skip connections that implement residual blocks. -3 means to add the feature maps output by the previous layer to those output by the layer three layers
  back. Linear activation means identity (no nonlinear activation of the result).
  
      [shortcut]
      from=-3           # Connect the layer three layers back to here.
      activation=linear


- Upsample: Bilinear upsampling of the previous layer using a particular stride

      [upsample]
      stride=2


- Route: The route layer deserves a bit of explanation. It has an attribute `layers`, which can have either one or two values.
  
      [route]
      layers = -4

      [route]
      layers = -1, 61    
  
  When the layers attribute has only one value, it outputs the feature maps of the layer indexed by the value. In our example, it is -4, so the layer will output
  the feature maps from the 4th layer backwards from the route layer.

  When layers has two values, it returns the concatenated feature maps of the layers indexed by its values. In our example it is -1, 61, so the layer will output
  feature maps from the previous layer (-1) and the 61st layer, concatenated along the channels (depth) dimension.
   
- YOLO:
 
      [yolo]
      mask = 0,1,2
      anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
      classes=80
      num=9
      jitter=.3
      ignore_thresh = .5
      truth_thresh = 1
      random=1
  
  Here we have a few important attributes:
  
  - anchors: describes the anchor boxes. The model contains 9 anchors, but only those in the `mask` are used.

  - mask: which anchor indices will be used in this YOLO layer
     
  - classes: number of object classes


### Create a network from the config file

We are going to follow the general approach of some of the GitHub contributors who have developed PyTorch tools
to deal with Darknet models. In the file `darknet.py`, there's a `parse_cfg` function. The function will read
the Darknet configuration file and store the blocks in a dictionary.

<img src="img/configfunc.JPG" title="configfunc" style="width: 640px;" />

We'll then create PyTorch NN Modules for each of the blocks in the Darknet configuration as implemented in the
`create_modules` function. Take a look at this function for more understanding.

### Convolutional block

<img src="img/convolutionalblock.JPG" title="covolutionalblock" style="width: 600px;" />

### Shortcut block

<img src="img/shortcutblock.JPG" title="shortcutblock" style="width: 600px;" />

### Upsample block

<img src="img/upsampleblock.JPG" title="upsampleblock" style="width: 600px;" />

### Route block

Why does it use an empty layer? The actual work will be done in the `forward()` function.

<img src="img/routeblock.JPG" title="routeblock" style="width: 600px;" />

### YOLO block

<img src="img/yoloblock.JPG" title="yoloblock" style="width: 600px;" />

### Using the code

OK, let's try it out. Depending on what you already have installed, you may need to run

    # apt install libgl1-mesa-glx

for the next step to run.

In [1]:
import darknet

blocks = darknet.parse_cfg("cfg/yolov3.cfg")
print(darknet.create_modules(blocks))

({'type': 'net', 'batch': '1', 'subdivisions': '1', 'width': '416', 'height': '416', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, ModuleList(
  (0): Sequential(
    (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_0): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (1): Sequential(
    (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (2): Sequential(
    (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    

### Darknet class

Let's make our own version of the `Darknet` class in `darknet.py`.

The class has two main functions:

1. `forward()`: forward propagation, following the instructions in the dictionary modules
2. `load_weights()`: load a set of pretrained weights into the network


In [2]:
from util import *

class MyDarknet(nn.Module):
    def __init__(self, cfgfile):
        super(MyDarknet, self).__init__()
        # load the config file and create our model
        self.blocks = darknet.parse_cfg(cfgfile)
        self.net_info, self.module_list = darknet.create_modules(self.blocks)
        
    def forward(self, x, CUDA:bool):
        modules = self.blocks[1:]
        outputs = {}   #We cache the outputs for the route layer
        
        write = 0
        # run forward propagation. Follow the instruction from dictionary modules
        for i, module in enumerate(modules):        
            module_type = (module["type"])
            
            if module_type == "convolutional" or module_type == "upsample":
                # do convolutional network
                x = self.module_list[i](x)
    
            elif module_type == "route":
                # concat layers
                layers = module["layers"]
                layers = [int(a) for a in layers]
    
                if (layers[0]) > 0:
                    layers[0] = layers[0] - i
    
                if len(layers) == 1:
                    x = outputs[i + (layers[0])]
    
                else:
                    if (layers[1]) > 0:
                        layers[1] = layers[1] - i
    
                    map1 = outputs[i + layers[0]]
                    map2 = outputs[i + layers[1]]
                    x = torch.cat((map1, map2), 1)
                
    
            elif  module_type == "shortcut":
                from_ = int(module["from"])
                # residual network
                x = outputs[i-1] + outputs[i+from_]
    
            elif module_type == 'yolo':        
                anchors = self.module_list[i][0].anchors
                #Get the input dimensions
                inp_dim = int (self.net_info["height"])
        
                #Get the number of classes
                num_classes = int (module["classes"])
        
                #Transform 
                # predict_transform is in util.py
                batch_size = x.size(0)
                stride =  inp_dim // x.size(2)
                grid_size = inp_dim // stride
                bbox_attrs = 5 + num_classes
                num_anchors = len(anchors)
                
                x = predict_transform(x, inp_dim, anchors, num_classes, CUDA)
                if not write:              #if no collector has been intialised. 
                    detections = x
                    write = 1
        
                else:       
                    detections = torch.cat((detections, x), 1)
        
            outputs[i] = x
        
        return detections


    def load_weights(self, weightfile):
        '''
        Load pretrained weight
        '''
        #Open the weights file
        fp = open(weightfile, "rb")
    
        #The first 5 values are header information 
        # 1. Major version number
        # 2. Minor Version Number
        # 3. Subversion number 
        # 4,5. Images seen by the network (during training)
        header = np.fromfile(fp, dtype = np.int32, count = 5)
        self.header = torch.from_numpy(header)
        self.seen = self.header[3]   
        
        weights = np.fromfile(fp, dtype = np.float32)
        
        ptr = 0
        for i in range(len(self.module_list)):
            module_type = self.blocks[i + 1]["type"]
    
            #If module_type is convolutional load weights
            #Otherwise ignore.
            
            if module_type == "convolutional":
                model = self.module_list[i]
                try:
                    batch_normalize = int(self.blocks[i+1]["batch_normalize"])
                except:
                    batch_normalize = 0
            
                conv = model[0]
                
                
                if (batch_normalize):
                    bn = model[1]
        
                    #Get the number of weights of Batch Norm Layer
                    num_bn_biases = bn.bias.numel()
        
                    #Load the weights
                    bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
        
                    bn_weights = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
        
                    bn_running_mean = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
        
                    bn_running_var = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
        
                    #Cast the loaded weights into dims of model weights. 
                    bn_biases = bn_biases.view_as(bn.bias.data)
                    bn_weights = bn_weights.view_as(bn.weight.data)
                    bn_running_mean = bn_running_mean.view_as(bn.running_mean)
                    bn_running_var = bn_running_var.view_as(bn.running_var)
        
                    #Copy the data to model
                    bn.bias.data.copy_(bn_biases)
                    bn.weight.data.copy_(bn_weights)
                    bn.running_mean.copy_(bn_running_mean)
                    bn.running_var.copy_(bn_running_var)
                
                else:
                    #Number of biases
                    num_biases = conv.bias.numel()
                
                    #Load the weights
                    conv_biases = torch.from_numpy(weights[ptr: ptr + num_biases])
                    ptr = ptr + num_biases
                
                    #reshape the loaded weights according to the dims of the model weights
                    conv_biases = conv_biases.view_as(conv.bias.data)
                
                    #Finally copy the data
                    conv.bias.data.copy_(conv_biases)
                    
                #Let us load the weights for the Convolutional layers
                num_weights = conv.weight.numel()
                
                #Do the same as above for weights
                conv_weights = torch.from_numpy(weights[ptr:ptr+num_weights])
                ptr = ptr + num_weights
                
                conv_weights = conv_weights.view_as(conv.weight.data)
                conv.weight.data.copy_(conv_weights)


### Test Forward Propagation

Let's propagate a single image through the network and see what we get.

In [3]:
!wget https://github.com/ayooshkathuria/pytorch-yolo-v3/raw/master/dog-cycle-car.png

--2022-05-13 20:48:19--  https://github.com/ayooshkathuria/pytorch-yolo-v3/raw/master/dog-cycle-car.png
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ayooshkathuria/pytorch-yolo-v3/master/dog-cycle-car.png [following]
--2022-05-13 20:48:19--  https://raw.githubusercontent.com/ayooshkathuria/pytorch-yolo-v3/master/dog-cycle-car.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347445 (339K) [image/png]
Saving to: ‘dog-cycle-car.png.1’

dog-cycle-car.png.1 100%[===================>] 339.30K  --.-KB/s    in 0.1s    

2022-05-13 20:48:20 (3.06 MB/s) - ‘dog-cycle-car.png.1’ saved [347445/3474

Here's code to load the image into memory and push it through the model:

In [3]:
import cv2
import torch

def get_test_input():
    img = cv2.imread("dog-cycle-car.png")
    img = cv2.resize(img, (416,416))          #Resize to the input dimension
    img_ =  img[:,:,::-1].transpose((2,0,1))  # BGR -> RGB | H X W C -> C X H X W 
    img_ = img_[np.newaxis,:,:,:]/255.0       #Add a channel at 0 (for batch) | Normalise
    img_ = torch.from_numpy(img_).float()     #Convert to float
    img_ = Variable(img_)                     # Convert to Variable
    return img_

Go ahead and try it (noting that the model hasn't been trained so we don't expect any correct result):

In [4]:
from util import *

model = MyDarknet("cfg/yolov3.cfg")
inp = get_test_input()
pred = model(inp, False)
print (pred)

tensor([[[1.7989e+01, 1.7215e+01, 1.7173e+02,  ..., 3.6508e-01,
          5.3973e-01, 4.8125e-01],
         [1.8091e+01, 1.6236e+01, 1.2679e+02,  ..., 5.1979e-01,
          3.9267e-01, 4.8417e-01],
         [1.5727e+01, 1.6022e+01, 2.2185e+02,  ..., 5.1345e-01,
          4.4406e-01, 5.3893e-01],
         ...,
         [4.1211e+02, 4.1190e+02, 1.4210e+01,  ..., 5.0629e-01,
          6.2061e-01, 4.7485e-01],
         [4.1192e+02, 4.1229e+02, 2.0387e+01,  ..., 5.1566e-01,
          4.6659e-01, 3.7516e-01],
         [4.1164e+02, 4.1201e+02, 4.0127e+01,  ..., 4.4963e-01,
          3.6770e-01, 3.9186e-01]]], grad_fn=<CatBackward>)


In [7]:
print(pred.shape)

torch.Size([1, 10647, 85])


### Understanding the output result

The result from prediction model will be $B(13\cdot 13 + 26\cdot 26 + 52 \cdot 52)3\cdot85$. Why? We have
- $B$: the number of images in the batch
- $13\cdot 13$: number of elements (grid cells) in the coarsest feature map
- $26\cdot 16$: number of elements (grid cells) in the medium scale feature map
- $52\cdot 52$: number of elements (grid cells) in the finest cale feature map
- $3$: the number of anchor boxes per grid cell
- $85$: number of bounding box attributes (4 for bounding box, 1 for objectness, 80 for the COCO classes)

### Download a pretrained weight file

Darknet stores weights as in this diagram:

<img src="img/weights.png" title="weight" style="width: 600px;" />

In [7]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2022-05-12 18:11:56--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  13.9MB/s    in 19s     

2022-05-12 18:12:16 (12.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



You can download the yolov3 weights from [here](https://drive.google.com/file/d/1T_rTXbBx8pO-2UeehUSP4R3UN7vGqcAG/view?usp=sharing)

In [ ]:
model.load_weights("yolov3.weights")

### Test with the sample image again

In [9]:
inp = get_test_input()
pred = model(inp, False)
print (pred)

tensor([[[8.5426e+00, 1.9015e+01, 1.1130e+02,  ..., 1.7306e-03,
          1.3874e-03, 9.2985e-04],
         [1.4105e+01, 1.8867e+01, 9.4014e+01,  ..., 5.9501e-04,
          9.2471e-04, 1.3085e-03],
         [2.1125e+01, 1.5269e+01, 3.5793e+02,  ..., 8.3609e-03,
          5.1067e-03, 5.8562e-03],
         ...,
         [4.1268e+02, 4.1069e+02, 3.7157e+00,  ..., 1.7185e-06,
          4.0955e-06, 6.5896e-07],
         [4.1132e+02, 4.1023e+02, 8.0353e+00,  ..., 1.3926e-05,
          3.2252e-05, 1.2076e-05],
         [4.1076e+02, 4.1318e+02, 4.9635e+01,  ..., 4.2174e-06,
          1.0794e-05, 1.8104e-05]]], grad_fn=<CatBackward>)


### From YOLO output tensor to *true* detections

In the prediction result, there are many results. We need to threshold them using the objectness score
output for each bounding box prediction. The `write_results` function in `util.py` does just that.

    def write_results(prediction, confidence, num_classes, nms_conf = 0.4)

- prediction: prediction result tensor returned from the YOLO model
- confidence: objectness score threshold to apply to the set of detections
- num_classes: number of classes to expect
- nms_conf: NMS IoU threshold

NMS stands for "non-maxima suppression." The basic idea is that if you have two predicted bounding
boxes that overlap each other significantly, you should throw away the box with the lower confidence
score. Overlap is measured by IoU (Intersection over Union), wich is just the ratio of the the area
of intersection of the two regions with the area of the union of the two regions:

$$ IoU(R_1,R_2) = \frac{|R_1 \cap R_2|}{|R_1 \cup R_2|}. $$

The default of 0.4 means if the intersection is 40% or more of the union, the two bounding boxes
are overlapping enough that only one of the detections should survive.

In [10]:
write_results(pred.detach(), 0.5, 80, nms_conf = 0.4)

tensor([[  0.0000,  61.5404, 100.8597, 307.2717, 303.1133,   0.9469,   0.9985,
           1.0000],
        [  0.0000, 253.8483,  66.1096, 378.0396, 118.0089,   0.9992,   0.8164,
           7.0000],
        [  0.0000,  71.0337, 163.2243, 175.7471, 382.2702,   0.9999,   0.9936,
          16.0000]])

### Show the resulting detections on top of an image

The model was trained on the COCO dataset, so download the class label file `coco.names`:

In [12]:
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/data/coco.names
!mkdir data
!mv coco.names data/coco.names

--2022-05-12 18:15:00--  https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2022-05-12 18:15:00 (17.9 MB/s) - ‘coco.names’ saved [625/625]



In [11]:
def load_classes(namesfile):
    fp = open(namesfile, "r")
    names = fp.read().split("\n")[:-1]
    return names

In [12]:
num_classes = 80
classes = load_classes("data/coco.names")
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


So we see that the three surviving bounding boxes above, outputting object types 1, 7, and 16, indicate a bicycle, a truck, and a dog.
Let's draw the detections on top of the input image for better visualization.

We'll use some code based on Kathuria's `detect.py`. You can download the original as

In [15]:
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/detect.py

--2021-02-06 01:53:13--  https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/detect.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7273 (7.1K) [text/plain]
Saving to: ‘detect.py’

detect.py           100%[===================>]   7.10K  --.-KB/s    in 0s      

2021-02-06 01:53:13 (33.2 MB/s) - ‘detect.py’ saved [7273/7273]



Here's our version. It will process the images in subdirectory `cocoimages` so let's make it and put our sample there:

In [15]:
!mkdir -p cocoimages
!cp dog-cycle-car.png cocoimages/

In [13]:
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
import argparse
import os 
import os.path as osp
from darknet import Darknet
import pickle as pkl
import pandas as pd
import random

images = "cocoimages"
batch_size = 4
confidence = 0.5
nms_thesh = 0.4
start = 0
CUDA = torch.cuda.is_available()

num_classes = 80
classes = load_classes("data/coco.names")

#Set up the neural network

print("Loading network.....")
model = MyDarknet("cfg/yolov3.cfg")
model.load_weights("yolov3.weights")
print("Network successfully loaded")

model.net_info["height"] = 416
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32

#If there's a GPU availible, put the model on GPU

if CUDA:
    model.cuda()

# Set the model in evaluation mode

model.eval()

read_dir = time.time()

# Detection phase

try:
    imlist = [osp.join(osp.realpath('.'), images, img) for img in os.listdir(images)]
except NotADirectoryError:
    imlist = []
    imlist.append(osp.join(osp.realpath('.'), images))
except FileNotFoundError:
    print ("No file or directory with the name {}".format(images))
    exit()
    
if not os.path.exists("des"):
    os.makedirs("des")

load_batch = time.time()
loaded_ims = [cv2.imread(x) for x in imlist]

im_batches = list(map(prep_image, loaded_ims, [inp_dim for x in range(len(imlist))]))
im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)


leftover = 0
if (len(im_dim_list) % batch_size):
    leftover = 1

if batch_size != 1:
    num_batches = len(imlist) // batch_size + leftover            
    im_batches = [torch.cat((im_batches[i*batch_size : min((i +  1)*batch_size,
                        len(im_batches))]))  for i in range(num_batches)]  

write = 0

if CUDA:
    im_dim_list = im_dim_list.cuda()
    
start_det_loop = time.time()
for i, batch in enumerate(im_batches):
    # Load the image 
    start = time.time()
    if CUDA:
        batch = batch.cuda()
    with torch.no_grad():
        prediction = model(Variable(batch), CUDA)

    prediction = write_results(prediction, confidence, num_classes, nms_conf = nms_thesh)

    end = time.time()

    if type(prediction) == int:

        for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
            im_id = i*batch_size + im_num
            print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
            print("{0:20s} {1:s}".format("Objects Detected:", ""))
            print("----------------------------------------------------------")
        continue

    prediction[:,0] += i*batch_size    #transform the atribute from index in batch to index in imlist 

    if not write:                      #If we have't initialised output
        output = prediction  
        write = 1
    else:
        output = torch.cat((output,prediction))

    for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
        im_id = i*batch_size + im_num
        objs = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]
        print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
        print("{0:20s} {1:s}".format("Objects Detected:", " ".join(objs)))
        print("----------------------------------------------------------")

    if CUDA:
        torch.cuda.synchronize()       
try:
    output
except NameError:
    print ("No detections were made")
    exit()

im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())

scaling_factor = torch.min(416/im_dim_list,1)[0].view(-1,1)

output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2

output[:,1:5] /= scaling_factor

for i in range(output.shape[0]):
    output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim_list[i,0])
    output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim_list[i,1])
    
output_recast = time.time()
class_load = time.time()
colors = [[255, 0, 0], [255, 0, 0], [255, 255, 0], [0, 255, 0], [0, 255, 255], [0, 0, 255], [255, 0, 255]]

draw = time.time()

def write(x, results):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results[int(x[0])]
    cls = int(x[-1])
    color = random.choice(colors)
    label = "{0}".format(classes[cls])
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img


list(map(lambda x: write(x, loaded_ims), output))

det_names = pd.Series(imlist).apply(lambda x: "{}/det_{}".format("des",x.split("/")[-1]))

list(map(cv2.imwrite, det_names, loaded_ims))

end = time.time()

print("SUMMARY")
print("----------------------------------------------------------")
print("{:25s}: {}".format("Task", "Time Taken (in seconds)"))
print()
print("{:25s}: {:2.3f}".format("Reading addresses", load_batch - read_dir))
print("{:25s}: {:2.3f}".format("Loading batch", start_det_loop - load_batch))
print("{:25s}: {:2.3f}".format("Detection (" + str(len(imlist)) +  " images)", output_recast - start_det_loop))
print("{:25s}: {:2.3f}".format("Output Processing", class_load - output_recast))
print("{:25s}: {:2.3f}".format("Drawing Boxes", end - draw))
print("{:25s}: {:2.3f}".format("Average time_per_img", (end - load_batch)/len(imlist)))
print("----------------------------------------------------------")


torch.cuda.empty_cache()

Loading network.....
Network successfully loaded
dog-cycle-car.png    predicted in  0.066 seconds
Objects Detected:    bicycle truck dog
----------------------------------------------------------
SUMMARY
----------------------------------------------------------
Task                     : Time Taken (in seconds)

Reading addresses        : 0.000
Loading batch            : 0.011
Detection (1 images)     : 0.269
Output Processing        : 0.000
Drawing Boxes            : 0.011
Average time_per_img     : 0.291
----------------------------------------------------------


Voila! You got the YOLO result in folder "des"

<img src="img/dogresult.png" title="weight" style="width: 600px;" />

### Training in YOLOv3

Now let's create a training script. We need to consider two components:
 - The dataset
 - The training procedure
 
#### Dataset

You can download the COCO dataset from [COCO dataset website](https://cocodataset.org/#home).
If you're running on the CSIM network, please use the copy that's in `/home2/mdailey/Datasets/coco/`.
If you want to create a dataset in this format yourself,
you must create an `images/` folder and an annotation file in the COCO JSON format.
To create the annotation file for your custom dataset,
there are many applications to help you, for example [makesense](https://www.makesense.ai/).

Put the folder name in your code:

In [9]:
path2data_train="/path/to/train/images/"
path2json_train="/path/to/train/coco_annotation.json"

path2data_val="/path/to/validate/images/"
path2json_val="/path/to/validate/coco_annotation.json"

In [10]:
# path2data_train="/home/alisa/fiftyone/coco-2017/train2017/data/"
# path2json_train="/home/alisa/fiftyone/coco-2017/annotations/instances_train2017.json"

# path2data_val="/home/alisa/fiftyone/coco-2017/val2017/data"
# path2json_val="/home/alisa/fiftyone/coco-2017/annotations/instances_val2017.json"


path2data_train="/home/alisa/fiftyone/coco-2017/train2017/data/"
path2json_train="/home/alisa/fiftyone/coco-2017/annotations/instances_train2017.json"

path2data_val="/home/alisa/fiftyone/coco-2017/val2017/data"
path2json_val="/home/alisa/fiftyone/coco-2017/annotations/instances_val2017.json"

img_size = 416

Next we create a custom dataset in the COCO format. The important thing is the annotation position. You need to create an annotation for each image in the YOLO format.
Thus, the class `CustomCoco` contains a `create_label()` function to convert bounding boxes in each class into corresponding YOLO labels.

In [11]:
# coding=utf-8
import os
import sys
from PIL import Image
import torch
import math
import cv2
import numpy as np
import torch
import random
from torchvision.datasets import CocoDetection

from typing import Any, Callable, Optional, Tuple
import json

ANCHORS = [
            [[12, 16], [19, 36], [40, 28]],
            [[36, 75], [76, 55], [72, 146]],
            [[142, 110], [192, 243], [459, 401]]
]

STRIDES = [8, 16, 32]

IP_SIZE = img_size
NUM_ANCHORS = 3
NUM_CLASSES = 80

with open('coco_cats.json') as js:
    data = json.load(js)["categories"]

cats_dict = {}
for i in range(0, 80):
    cats_dict[str(data[i]['id'])] = i



class CustomCoco(CocoDetection):
    def __init__(
            self,
            root: str,
            annFile: str,
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None,
            transforms: Optional[Callable] = None,
    ) -> None:
        super(CocoDetection, self).__init__(root, transforms, transform, target_transform)
        from pycocotools.coco import COCO
        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))


    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image, target). target is the object returned by ``coco.loadAnns``.
        """
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        target = coco.loadAnns(ann_ids)

        path = coco.loadImgs(img_id)[0]['file_name']

        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        img = np.array(img)

        category_ids = list(obj['category_id'] for obj in target)
        bboxes = list(obj['bbox'] for obj in target)
  
        if self.transform is not None:
            bboxes = list(obj['bbox'] for obj in target)
            category_ids = list(obj['category_id'] for obj in target)
            transformed = self.transform(image=img, bboxes=bboxes, category_ids=category_ids)
            img = transformed['image'],
            bboxes = torch.Tensor(transformed['bboxes'])
            cat_ids = torch.Tensor(transformed['category_ids'])
            labels, bboxes = self.__create_label(bboxes, cat_ids.type(torch.IntTensor))

        return img, labels, bboxes

    def __len__(self) -> int:
        return len(self.ids)

    def __create_label(self, bboxes, class_inds):
        """
        Label assignment. For a single picture all GT box bboxes are assigned anchor.
        1、Select a bbox in order, convert its coordinates("xyxy") to "xywh"; and scale bbox'
           xywh by the strides.
        2、Calculate the iou between the each detection layer'anchors and the bbox in turn, and select the largest
            anchor to predict the bbox.If the ious of all detection layers are smaller than 0.3, select the largest
            of all detection layers' anchors to predict the bbox.
        Note :
        1、The same GT may be assigned to multiple anchors. And the anchors may be on the same or different layer.
        2、The total number of bboxes may be more than it is, because the same GT may be assigned to multiple layers
        of detection.
        """
        # print("Class indices: ", class_inds)
        bboxes = np.array(bboxes)
        class_inds = np.array(class_inds)
        anchors = ANCHORS # all the anchors
        strides = np.array(STRIDES) # list of strides
        train_output_size = IP_SIZE / strides # image with different scales
        anchors_per_scale = NUM_ANCHORS # anchor per scale

        label = [
            np.zeros(
                (
                    int(train_output_size[i]),
                    int(train_output_size[i]),
                    anchors_per_scale,
                    5 + NUM_CLASSES,
                )
            )
            for i in range(3)
        ]
        # 150 bounding box ground truths per scale
        bboxes_xywh = [
            np.zeros((150, 4)) for _ in range(3)
        ]  # Darknet the max_num is 30
        bbox_count = np.zeros((3,))

        for i in range(len(bboxes)):
            bbox_coor = bboxes[i][:4]
            bbox_class_ind = cats_dict[str(class_inds[i])]

            # onehot
            one_hot = np.zeros(NUM_CLASSES, dtype=np.float32)
            one_hot[bbox_class_ind] = 1.0
            # one_hot_smooth = dataAug.LabelSmooth()(one_hot, self.num_classes)

            # convert "xyxy" to "xywh"
            bbox_xywh = np.concatenate(
                [
                    (0.5 * bbox_coor[2:] + bbox_coor[:2]) ,
                    bbox_coor[2:],
                ],
                axis=-1,
            )
            
            bbox_xywh_scaled = (
                1.0 * bbox_xywh[np.newaxis, :] / strides[:, np.newaxis]
            )

            iou = []
            exist_positive = False
            for i in range(3):
                anchors_xywh = np.zeros((anchors_per_scale, 4))
                anchors_xywh[:, 0:2] = (
                    np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.int32) + 0.5
                )  # 0.5 for compensation

                # assign all anchors 
                anchors_xywh[:, 2:4] = anchors[i]

                iou_scale = iou_xywh_numpy(
                    bbox_xywh_scaled[i][np.newaxis, :], anchors_xywh
                )
                iou.append(iou_scale)
                iou_mask = iou_scale > 0.3

                if np.any(iou_mask):
                    xind, yind = np.floor(bbox_xywh_scaled[i, 0:2]).astype(
                        np.int32
                    )

                    label[i][yind, xind, iou_mask, 0:4] = bbox_xywh * strides[i]
                    label[i][yind, xind, iou_mask, 4:5] = 1.0
                    label[i][yind, xind, iou_mask, 5:] = one_hot

                    bbox_ind = int(bbox_count[i] % 150)  # BUG : 150为一个先验值,内存消耗大
                    bboxes_xywh[i][bbox_ind, :4] = bbox_xywh * strides[i]
                    bbox_count[i] += 1

                    exist_positive = True

            if not exist_positive:
                # check if a ground truth bb have the best anchor with any scale
                best_anchor_ind = np.argmax(np.array(iou).reshape(-1), axis=-1)
                best_detect = int(best_anchor_ind / anchors_per_scale)
                best_anchor = int(best_anchor_ind % anchors_per_scale)

                xind, yind = np.floor(
                    bbox_xywh_scaled[best_detect, 0:2]
                ).astype(np.int32)

                label[best_detect][yind, xind, best_anchor, 0:4] = bbox_xywh * strides[best_detect]
                label[best_detect][yind, xind, best_anchor, 4:5] = 1.0
                # label[best_detect][yind, xind, best_anchor, 5:6] = bbox_mix
                label[best_detect][yind, xind, best_anchor, 5:] = one_hot 

                bbox_ind = int(bbox_count[best_detect] % 150)
                bboxes_xywh[best_detect][bbox_ind, :4] = bbox_xywh * strides[best_detect]
                bbox_count[best_detect] += 1

        flatten_size_s = int(train_output_size[2]) * int(train_output_size[2]) * anchors_per_scale
        flatten_size_m = int(train_output_size[1]) * int(train_output_size[1]) * anchors_per_scale
        flatten_size_l = int(train_output_size[0]) * int(train_output_size[0]) * anchors_per_scale

        label_s = torch.Tensor(label[2]).view(1, flatten_size_s, 5 + NUM_CLASSES).squeeze(0)
        label_m = torch.Tensor(label[1]).view(1, flatten_size_m, 5 + NUM_CLASSES).squeeze(0)
        label_l = torch.Tensor(label[0]).view(1, flatten_size_l, 5 + NUM_CLASSES).squeeze(0)

        bboxes_s = torch.Tensor(bboxes_xywh[2])
        bboxes_m = torch.Tensor(bboxes_xywh[1])
        bboxes_l = torch.Tensor(bboxes_xywh[0])

        # label_sbbox, label_mbbox, label_lbbox = label
        sbboxes, mbboxes, lbboxes = bboxes_xywh
        # print("label")
        labels = torch.cat([label_l, label_m, label_s], 0)
        bboxes = torch.cat([bboxes_l, bboxes_m, bboxes_s], 0)
        return labels, bboxes

Note that we still need two important functions for training:
- iou_xywh_numpy: calculate the IoU (intersection over union) between two sets of bounding boxes `boxes1` and `boxes2`. This function is used in the `CustomCoco` class.
- CIOU_xywh_torch: CIOU or "Complete IOU" combines how well two boxes overlap, the distance between their centers, and the similarity of their aspect ratios.

In [12]:
def iou_xywh_numpy(boxes1, boxes2):
    boxes1 = np.array(boxes1)
    boxes2 = np.array(boxes2)
    # print(boxes1, boxes2)

    boxes1_area = boxes1[..., 2] * boxes1[..., 3]
    boxes2_area = boxes2[..., 2] * boxes2[..., 3]

    boxes1 = np.concatenate([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                                boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2 = np.concatenate([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                                boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)

    left_up = np.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down = np.minimum(boxes1[..., 2:], boxes2[..., 2:])

    inter_section = np.maximum(right_down - left_up, 0.0)
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    union_area = boxes1_area + boxes2_area - inter_area
    IOU = 1.0 * inter_area / union_area
    return IOU


def CIOU_xywh_torch(boxes1,boxes2):
    '''
    cal CIOU of two boxes or batch boxes
    :param boxes1:[xmin,ymin,xmax,ymax] or
                [[xmin,ymin,xmax,ymax],[xmin,ymin,xmax,ymax],...]
    :param boxes2:[xmin,ymin,xmax,ymax]
    :return:
    '''
    # cx cy w h->xyxy
    boxes1 = torch.cat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], dim=-1)
    boxes2 = torch.cat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], dim=-1)

    boxes1 = torch.cat([torch.min(boxes1[..., :2], boxes1[..., 2:]),
                        torch.max(boxes1[..., :2], boxes1[..., 2:])], dim=-1)
    boxes2 = torch.cat([torch.min(boxes2[..., :2], boxes2[..., 2:]),
                        torch.max(boxes2[..., :2], boxes2[..., 2:])], dim=-1)

    # (x2 minus x1 = width)  * (y2 - y1 = height)
    boxes1_area = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
    boxes2_area = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])

    # upper left of the intersection region (x,y)
    inter_left_up = torch.max(boxes1[..., :2], boxes2[..., :2])

    # bottom right of the intersection region (x,y)
    inter_right_down = torch.min(boxes1[..., 2:], boxes2[..., 2:])

    # if there is overlapping we will get (w,h) else set to (0,0) because it could be negative if no overlapping
    inter_section = torch.max(inter_right_down - inter_left_up, torch.zeros_like(inter_right_down))
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    union_area = boxes1_area + boxes2_area - inter_area
    ious = 1.0 * inter_area / union_area

    # cal outer boxes
    outer_left_up = torch.min(boxes1[..., :2], boxes2[..., :2])
    outer_right_down = torch.max(boxes1[..., 2:], boxes2[..., 2:])
    outer = torch.max(outer_right_down - outer_left_up, torch.zeros_like(inter_right_down))
    outer_diagonal_line = torch.pow(outer[..., 0], 2) + torch.pow(outer[..., 1], 2)

    # cal center distance
    # center x center y
    boxes1_center = (boxes1[..., :2] +  boxes1[...,2:]) * 0.5
    boxes2_center = (boxes2[..., :2] +  boxes2[...,2:]) * 0.5

    # euclidean distance
    # x1-x2 square 
    center_dis = torch.pow(boxes1_center[...,0]-boxes2_center[...,0], 2) +\
                 torch.pow(boxes1_center[...,1]-boxes2_center[...,1], 2)

    # cal penalty term
    # cal width,height
    boxes1_size = torch.max(boxes1[..., 2:] - boxes1[..., :2], torch.zeros_like(inter_right_down))
    boxes2_size = torch.max(boxes2[..., 2:] - boxes2[..., :2], torch.zeros_like(inter_right_down))
    v = (4 / (math.pi ** 2)) * torch.pow(
            torch.atan((boxes1_size[...,0]/torch.clamp(boxes1_size[...,1],min = 1e-6))) -
            torch.atan((boxes2_size[..., 0] / torch.clamp(boxes2_size[..., 1],min = 1e-6))), 2)

    alpha = v / (1-ious+v)

    #cal ciou
    cious = ious - (center_dis / outer_diagonal_line + alpha*v)

    return cious

#### Training

After finished your custom dataset code, let's train your data.

In [13]:
from __future__ import division
import time
import os
import os.path as osp
import numpy as np
import cv2
import pickle as pkl
import pandas as pd
import random
from copy import copy, deepcopy
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Subset
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import datasets, models, transforms

from util import *

import albumentations as A

Select the device

In [14]:
# Set device to GPU or CPU
gpu = "0"
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() else "cpu")

#### Load dataset
Make image transform and load train dataset. We have used the albumentations class which can transform box parameters automatically, but because we want to test the training coco in yolov3, the transform of training is only resize to $416 \times 416$.

In [15]:
train_transform = A.Compose([
    #A.SmallestMaxSize(256),
    A.Resize(img_size, img_size),
    # A.RandomCrop(width=224, height=224),
    # A.HorizontalFlip(p=0.5),
    # A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

eval_transform = A.Compose([
    A.Resize(img_size, img_size),
    #A.SmallestMaxSize(256),
    #A.CenterCrop(width=224, height=224),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

In [16]:
BATCH_SIZE = 10
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = Subset(CustomCoco(root = path2data_train,
                                annFile = path2json_train, transform=train_transform), list(range(0,20)))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=0, collate_fn=collate_fn)

loading annotations into memory...
Done (t=8.18s)
creating index...
index created!


#### Load Yolov3 model

If you want to train the model with pretrained network, load weights from the example above.

In [17]:
print("Loading network.....")
model = MyDarknet("cfg/yolov3.cfg")
# load pretrained
# model.load_weights("yolov3.weights")
print("Network successfully loaded")

Loading network.....
Network successfully loaded


#### Training code

Create the training code. The important part of yolo training is you must convert and calculate the iou correctly.

In [18]:
def run_training(model, optimizer, dataloader, device, img_size, n_epoch, every_n_batch, every_n_epoch, ckpt_dir):
    losses = None
    for epoch_i in range(n_epoch):
        running_loss = 0.0
        for inputs, labels, bboxes in dataloader:
            inputs = torch.from_numpy(np.array(inputs)).squeeze(1).permute(0,3,1,2).float()
            inputs = inputs.to(device)
            labels = torch.stack(labels).to(device)
            #print(inputs.shape)

            running_corrects = 0

            # zero the parameter gradients
            # it uses for update training weights
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs, True)

                pred_xywh = outputs[..., 0:4] / img_size
                pred_conf = outputs[..., 4:5]
                pred_cls = outputs[..., 5:]

                label_xywh = labels[..., :4] / img_size

                label_obj_mask = labels[..., 4:5]
                label_noobj_mask = (1.0 - label_obj_mask)
                lambda_coord = 0.001
                lambda_noobj = 0.05
                label_cls = labels[..., 5:]
                loss = nn.MSELoss()
                loss_bce = nn.BCELoss()

                loss_coord = lambda_coord * label_obj_mask * loss(input=pred_xywh, target=label_xywh)
                loss_conf = (label_obj_mask * loss_bce(input=pred_conf, target=label_obj_mask)) + \
                            (lambda_noobj * label_noobj_mask * loss_bce(input=pred_conf, target=label_obj_mask))
                loss_cls = label_obj_mask * loss_bce(input=pred_cls, target=label_cls)

                loss_coord = torch.sum(loss_coord)
                loss_conf = torch.sum(loss_conf)
                loss_cls = torch.sum(loss_cls)

                # print(pred_xywh.shape, label_xywh.shape)

                ciou = CIOU_xywh_torch(pred_xywh, label_xywh)
                # print(ciou.shape)
                ciou = ciou.unsqueeze(-1)
                # print(ciou.shape)
                # print(label_obj_mask.shape)
                loss_ciou = torch.sum(label_obj_mask * (1.0 - ciou))
                # print(loss_coord)
                loss =  loss_ciou +  loss_conf + loss_cls
                loss.backward()
                optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                # print('Running loss')
                # print(loss_coord, loss_conf, loss_cls)
        epoch_loss = running_loss / 750
        print(epoch_loss)
        print('End Epoch')

Let's train it!

In [19]:
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epoch = 5
img_size = 416
save_every_batch = False
save_every_epoch = True
ckpt_dir = "../checkpoints"


run_training(model, optimizer, train_dataloader, device,
                 img_size,
                 n_epoch,
                 save_every_batch,
                 save_every_epoch,
                 ckpt_dir)

114.36709635416666
End Epoch
87.42002604166667
End Epoch
63.589274088541664
End Epoch
48.14430826822917
End Epoch
39.50013671875
End Epoch


## YOLOv7

The YOLO (You Only Look Once) v7 model is the latest in the family of YOLO models released in 2022.

The YOLOv7 authors sought to set the state of the art in object detection by creating a network architecture that would predict bounding boxes more accurately than its peers at similar inference speeds.

<img src="img/yolov7_eval.png" title="weight" style="width: 600px;" />

In order to achieve these results, the YOLOv7 authors made a number of changes to the YOLO network and training routines.

### Extended Efficient Layer Aggregation

YOLOv7 is built on previous research on this topic, taking into account the amount of memory required to keep layers in memory as well as the distance required for a gradient to back-propagate through the layers. The shorter the gradient, the more powerful their network's learning ability. E-ELAN, an extended version of the ELAN computational block, is the final layer aggregation they select.

<img src="img/Efficient_layer_aggregation.png" title="weight" style="width: 600px;" />


### Model Scaling Techniques

Object detection models typically take into account the depth of the network, the width of the network, and the resolution on which the network is trained. The authors of YOLOv7 scale the network depth and width simultaneously while concatenating layers together. According to ablation studies, this technique keeps the model architecture optimal while scaling for different sizes.

<img src="img/image-37.png" title="weight" style="width: 600px;" />

### Re-parameterization Planning

Averaging a set of model weights is used in re-parameterization techniques to create a model that is more robust to the general patterns that it is attempting to model. There has recently been a focus in research on module level re-parameterization, where each component of the network has its own re-parameterization strategy.

YOLOv7 uses gradient flow propagation paths to determine which network modules should and should not use re-parameterization strategies.

<img src="img/image-38.png" title="weight" style="width: 600px;" />

### Auxiliary Head Coarse-to-Fine

The YOLO network head makes the final network predictions, but because it is so far downstream in the network, it may be advantageous to add an auxiliary head somewhere in the middle.

Because there is less network between the auxiliary head and the prediction, it does not train as efficiently as the final head, so the YOLOv7 authors experiment with different levels of supervision for this head, settling on a coarse-to-fine definition in which supervision is passed back from the lead head at different granularities.

<img src="img/aux_head.png" title="weight" style="width: 600px;" />

### YOLOv7-mask 

The integration of YOLOv7 with BlendMask is used to perform instance segmentation. Therefore, the YOLOv7 object detection model was fine-tuned on the MS COCO instance segmentation dataset and trained for 30 epochs. It achieves state-of-the-art real-time instance segmentation results.

<img src="img/yolov7-mask-architecture.webp" title="weight" style="width: 300px;" />

### YOLOv7-pose 

The integration of YOLOv7 with YOLO-Pose allows keypoint detection for Pose Estimation. The authors fine-tuned a YOLOv7-W6 people detection model on the MS COCO keypoint detection dataset and achieved state-of-the-art real-time pose estimation performance.

<img src="img/yolov7-pose-architecture-259x350.webp" title="weight" style="width: 200px;" />

### Conclusion of performance

YOLOv7 significantly improves real-time object detection accuracy while lowering inference costs. As previously demonstrated in the benchmarks, YOLOv7 can effectively reduce about 40% of the parameters and 50% of the computation of state-of-the-art real-time object detections, while achieving faster inference speed and higher detection accuracy.

YOLOv7, in general, provides a faster and stronger network architecture with a more effective feature integration method, more accurate object detection performance, a more robust loss function, and increased label assignment and model training efficiency.

As a result, YOLOv7 requires significantly less computing power than other deep learning models. Without any pre-trained weights, it can be trained much faster on small datasets.

### The YOLOv7 Codebase

The Y[OLOv7 GitHub repository](https://github.com/WongKinYiu/yolov7) contains all of the code you need to get started training YOLOv7 on your custom data.

You can train the model by use this command.

In [ ]:
python train.py --workers 8 --device 0 --batch-size 32 --data data/coco.yaml --img 640 640 --cfg cfg/training/yolov7.yaml --weights '' --name yolov7 --hyp data/hyp.scratch.p5.yaml 

# New Homework (Will release on 3 Feb 2023)

## Independent exercise: YOLOR
### Part I: Inference (due next week)

In the lab, we saw how the Darknet configuration file for YOLOv3 could be read in Python and mapped
to PyTorch modules.

For your independent work do the same thing for YOLOv4. Download the `yolov4.cfg` file
from the [YOLOv4 GitHub repository](https://github.com/AlexeyAB/darknet) and modify your
`MyDarknet` class and utility code (`darknet.py`, `util.py`) as
necessary to map the structures to PyTorch.

The changes you'll have to make:

1. Implement the mish activation function
2. Add an option for a maxpool layer in the `create_modules` function and in your model's `forward()` method.
3. Enable a `[route]` module to concatenate more than two previous layers
4. Load the pre-trained weights [provided by the authors](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights)
4. Scale inputs to 608$\times$608 and make sure you're passing input channels in RGB order, not OpenCV's BGR order.

### Part II: Training (due in two weeks)

Train the YOLOv4 model on the COCO dataset (or another dataset if you have one available).
Here the purpose is not to get the best possible model (that would require implementing all
of the "bag of freebies" training tricks described in the paper), but just some of them, to
get a feel for their importance.

1. Get a set of ImageNet pretrained weights for CSPDarknet53 [from the Darknet GitHub repository](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/csdarknet53-omega_final.weights)
2. Add a method to load the pretrained weights into the backbone portion of your PyTorch YOLOv4 model.
3. Implement a basic `train_yolo` function similar to the `train_model` function you developed in previous
   labs for classifiers that preprocesses the input with basic augmentation transformations, converts the
   anchor-relative outputs to bounding box coordinates, computes MSE loss for the bounding box coordinates,
   backpropagates the loss, and takes a step for the optimizer. Use the recommended IoU thresholds to determine
   which predicted bounding boxes to include in the loss. You will find many examples of how to do this
   online.
4. Train your model on COCO. Training on the full dataset to completion would take several days, so you can stop early after verifying
   the model is learning in the first few epochs.
5. Compute mAP for your model on the COCO validation set.
6. Implement the CIoU loss function and observe its effect on mAP.
7. (Optional) Train on COCO to completion and see how close you can get to the mAP reported in the paper.

There is some useful information on working with the COCO dataset as a
Torchvision Dataset in [this blog](https://medium.com/howtoai/pytorch-torchvision-coco-dataset-b7f5e8cad82).
For your work on this lab, the instructor will place the entire COCO training and validation datasets on a shared network drive for you to access
so that we don't use resources for multiple copies of the dataset. Once you have access to the dataset you can use the dataset easily: